In [1]:
import sys
import numpy as np
from random import randint
import random
import math
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
import operator
import sklearn.metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.naive_bayes import GaussianNB as GNB
from pandas import DataFrame, read_csv
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
from deap import creator, base, tools, algorithms

In [2]:
file1 = '../../data/safecity/binary/groping/train.csv'
df = pd.read_csv(file1)
print(df.head())

corpus = list(df["Description"])
target = list(df["Category"])

                                         Description  Category
0  Was walking along crowded street, holding mums...         1
1  This incident took place in the evening.I was ...         0
2  I WAS WAITING FOR THE BUS. A MAN CAME ON A BIK...         0
3                 Incident happened inside the train         0
4  I witnessed an incident when a chain was bruta...         0


In [3]:
# infinity = float("inf") #sentinel maximum value
# m = 70 #no of bats (50-70)
# T = 100 # no of iterations(100)

# bat_pos = defaultdict(list)
# bat_vel = defaultdict(list)
# bat_loudness = []
# bat_rate = []
# bat_fitness = []
# bat_frequency = []

# global_bat_pos = []
# init_rate_vector = []
# feature_vector = []

# #hyperparameters
# alpha = 0.9
# gamma = 0.9
# epsilon = random.uniform(-1,1)
# fmin=0
# fmax =1

In [4]:
#do once
# nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
vectorizer = TfidfVectorizer(ngram_range=(1, 1), stop_words=stop_words)
X = vectorizer.fit_transform(corpus)
print(X.shape)
print(vectorizer.get_feature_names()[0:50])
print(len(vectorizer.get_feature_names()))

X_train = X
Y_train = target

file2 = '../../data/safecity/binary/groping/dev.csv'
df_dev = pd.read_csv(file2)
print(df_dev.head())
corpus_dev = list(df_dev["Description"])
target_dev = list(df_dev["Category"])
X_dev = vectorizer.transform(corpus_dev)
print(X_dev.shape)
print(vectorizer.get_feature_names()[0:50])
print(len(vectorizer.get_feature_names()))
Y_dev = target_dev

file3 = '../../data/safecity/binary/groping/test.csv'
df_test = pd.read_csv(file3)
print(df_test.head())
corpus_test = list(df_test["Description"])
target_test = list(df_test["Category"])
X_test = vectorizer.transform(corpus_test)
print(X_test.shape)
print(vectorizer.get_feature_names()[0:50])
print(len(vectorizer.get_feature_names()))
Y_test = target_test

(7201, 8504)
['00', '000', '00am', '00pm', '01', '03', '0300', '05', '06', '0740', '0800am', '0830', '08826862360', '0900', '0930', '0f', '10', '100', '1000', '1000am', '1011am', '1012', '1015', '1030', '1030am', '1030ish', '1030pm', '1050', '10pm', '10th', '11', '1115', '1130', '1139', '1150', '11am', '11h', '11th', '12', '1213', '1214', '1230', '1245', '12th', '12yearold', '12years', '12yrs', '13', '1314', '13th']
8504
                                         Description  Category
0  Buses approaching to this place is highly unsafe.         1
1                        a man was commenting at me.         0
2                                    in a share auto         1
3  I was coming out of a club at night with a few...         0
4  One of my friends was molested in the crowd. T...         1
(990, 8504)
['00', '000', '00am', '00pm', '01', '03', '0300', '05', '06', '0740', '0800am', '0830', '08826862360', '0900', '0930', '0f', '10', '100', '1000', '1000am', '1011am', '1012', '1015', '10

In [5]:
z1, z2 = X_train, X_dev
print(z1.shape, z2.shape)
n = z1.shape[1]

(7201, 8504) (990, 8504)


In [6]:
#auxillary  functions
def extract_subset(z,cols):
    return z[:,cols]

In [7]:
def svm_calc_fitness(x, mode=None): # SVM classifier
    if mode in (None, 'dev'): X_eval, Y_eval = X_dev, Y_dev
    elif mode == 'test': X_eval, Y_eval = X_test, Y_test
    elif mode == 'train': X_eval,  Y_eval = X_train, Y_train
        
    for i in range(0,len(x)):
        x[i] = x[i] > 0.5
    cols = []
    for j in range(0, n):
        if x[j] == 1:
            cols.append(j)
    z1_prime = extract_subset(X_train, cols)
    z2_prime = extract_subset(X_eval, cols)
    clf = svm.LinearSVC(C=1.0)
    clf.fit(z1_prime, Y_train)
    y_pred = clf.predict(z2_prime)
    
    acc = sklearn.metrics.accuracy_score(Y_eval,y_pred)
    pre = sklearn.metrics.precision_score(Y_eval,y_pred)
    fscore = sklearn.metrics.f1_score(Y_eval,y_pred)
    recall = sklearn.metrics.recall_score(Y_eval,y_pred)
    return acc,pre,fscore,recall

In [8]:
def rf_calc_fitness(x, mode=None): # RF classifier
    if mode in (None, 'dev'): X_eval, Y_eval = X_dev, Y_dev
    elif mode == 'test': X_eval, Y_eval = X_test, Y_test
    elif mode == 'train': X_eval,  Y_eval = X_train, Y_train
        
    for i in range(0,len(x)):
        x[i] = x[i] > 0.5
    cols = []
    for j in range(0, n):
        if x[j] == 1:
            cols.append(j)
    z1_prime = extract_subset(X_train, cols)
    z2_prime = extract_subset(X_eval, cols)
    clf = RandomForestClassifier(n_jobs=2,random_state=0)
    clf.fit(z1_prime, Y_train)
    y_pred = clf.predict(z2_prime)
    
    acc = sklearn.metrics.accuracy_score(Y_eval,y_pred)
    pre = sklearn.metrics.precision_score(Y_eval,y_pred)
    fscore = sklearn.metrics.f1_score(Y_eval,y_pred)
    recall = sklearn.metrics.recall_score(Y_eval,y_pred)
    return acc,pre,fscore,recall

In [9]:
def knn_calc_fitness(x, mode=None): # KNN classifier
    if mode in (None, 'dev'): X_eval, Y_eval = X_dev, Y_dev
    elif mode == 'test': X_eval, Y_eval = X_test, Y_test
    elif mode == 'train': X_eval,  Y_eval = X_train, Y_train

    for i in range(0,len(x)):
        x[i] = x[i] > 0.5
    cols = []
    for j in range(0, n):
        if x[j] == 1:
            cols.append(j)
    z1_prime = extract_subset(X_train, cols)
    z2_prime = extract_subset(X_eval, cols)
    clf = KNN(3,weights='uniform')
    clf.fit(z1_prime, Y_train)
    y_pred = clf.predict(z2_prime)
    
    acc = sklearn.metrics.accuracy_score(Y_eval,y_pred)
    pre = sklearn.metrics.precision_score(Y_eval,y_pred)
    fscore = sklearn.metrics.f1_score(Y_eval,y_pred)
    recall = sklearn.metrics.recall_score(Y_eval,y_pred)
    return acc,pre,fscore,recall

In [10]:
def nb_calc_fitness(x, mode=None): # Gaussian NB (Naive Bayes) classifier
    if mode in (None, 'dev'): X_eval, Y_eval = X_dev, Y_dev
    elif mode == 'test': X_eval, Y_eval = X_test, Y_test
    elif mode == 'train': X_eval,  Y_eval = X_train, Y_train
    
    for i in range(0,len(x)):
        x[i] = x[i] > 0.5
    cols = []
    for j in range(0, n):
        if x[j] == 1:
            cols.append(j)
    z1_prime = extract_subset(X_train, cols)
    z2_prime = extract_subset(X_eval, cols)
    clf = GNB()
    clf.fit(z1_prime, Y_train)
    y_pred = clf.predict(z2_prime)
    
    acc = sklearn.metrics.accuracy_score(Y_eval,y_pred)
    pre = sklearn.metrics.precision_score(Y_eval,y_pred)
    fscore = sklearn.metrics.f1_score(Y_eval,y_pred)
    recall = sklearn.metrics.recall_score(Y_eval,y_pred)
    return acc,pre,fscore,recall

In [11]:
def initialise(m,n):
    for i in range(0,m):
        for j in range(0,n):
            x = randint(0,1)
            bat_pos[i].append(x)
            bat_vel[i].append(0)

        init_rate_vector.append(random.uniform(0, 1))

        bat_frequency.append(init_rate_vector[i])
        bat_loudness.append(random.uniform(1,2))
        bat_rate.append(init_rate_vector[i])
        bat_fitness.append(-infinity)

In [12]:
# def core():
#     initialise(m,n)
#     global_fit = -infinity
#     for t in range (0,T):
#         print("T:" , t)
#         for i in range(0,m):
#             accuracy,pre,fscore,recall = svm_calc_fitness(bat_pos[i])
#             '''
#             with open("dumpPerformanceBat_svm_kpi.txt", "a") as text_file:
#                 text_file.write("{} {} {} {} {}".format(accuracy,pre,fscore,recall,global_fit) + "\n")
#             with open("dumpPerformanceBat_svm.txt", "a") as text_file:
#                 text_file.write("{} {} {} {}".format(t,accuracy,bat_pos[i].count(1),global_fit)+"\n")
#             #print("t={} accuracy={} #features={} global_fit={}".format(t,accuracy,bat_pos[i].count(1),global_fit))
#             '''
#             acc = accuracy

#             rand = randint(0,1)
#             if rand < bat_loudness[i] and acc > bat_fitness[i]:
#                 bat_fitness[i]=acc
#                 bat_loudness[i]= alpha * bat_loudness[i]
#                 bat_rate[i]= init_rate_vector[i] * (1 - math.exp(-(gamma * t)))

#         maxindex,maxfit = max(enumerate(bat_fitness), key=operator.itemgetter(1))

#         if maxfit>global_fit :
#             global_fit = maxfit
#             global_bat_pos = bat_pos[maxindex]

#         for i in range(0,m): # for each bat
#             beta = random.uniform(0,1)
#             rand = random.uniform(0,1)
#             AvgA = sum(bat_loudness)/len(bat_loudness)

#             if rand > bat_rate[i] :
#                 for j in range(0,n):
#                     bat_pos[i][j] = bat_pos[i][j] + ( epsilon * AvgA )
#                     sigma = randint(0,1)
#                     if sigma < (1/(1+math.exp(-bat_pos[i][j]))) :
#                         bat_pos[i][j]=1
#                     else :
#                         bat_pos[i][j]= 0

#             rand = random.uniform(0,1)
#             if rand < bat_loudness[i] and bat_fitness[i] < global_fit :
#                 for j in range (0,n):
#                     bat_frequency[i] = fmin + (fmax-fmin)* beta
#                     bat_vel[i][j] = bat_vel[i][j] + (global_bat_pos[j]-bat_pos[i][j]) * bat_frequency[i]
#                     bat_pos[i][j] = bat_pos[i][j] + bat_vel[i][j]
#                     sigma = random.uniform(0,1)
#                     if sigma < (1 / (1 + math.exp(-bat_pos[i][j]))) :
#                         bat_pos[i][j]=1
#                     else :
#                         bat_pos[i][j]=0
#         with open("dumpPerformanceBat_rf.txt", "a") as text_file:
#             text_file.write("{} {} {} ".format(t,global_bat_pos.count(1),global_fit)+"\n")
        


#     feature_vector = global_bat_pos
#     return feature_vector

In [13]:
# def BAT_algo():
#     feature_vector = core()
#     return feature_vector

In [14]:
# f = BAT_algo()
 

In [15]:
class GeneticAlgorithm():
    
    def __init__(self, X_train, X_dev, Y_train, Y_dev, number_bits_to_mutate = 25.0):
        
        
        self.toolbox = base.Toolbox()
        self.ind_mut_pb = float(number_bits_to_mutate / X_train.shape[1])
        self.NGEN = 100
        self.MUTPB = 0.2
        self.CXPB = 0.5
        self.POPULATION_SIZE = 70
        self.NUMBER_TO_SELECT_FROM_POP = 35
        self.init_pop = list()
        self.X_train = X_train
        self.X_dev = X_dev
        self.Y_train = Y_train
        self.Y_dev = Y_dev
        
    
    def initialise_toolbox(self):
        print('now initializing toolbox GA1')
        # I want to maximize session overlap as well as length
        creator.create("FitnessMax", base.Fitness, weights=(1.0,))
        # representing individual as a list
        creator.create("Individual", list, fitness=creator.FitnessMax)
        # individual will be a list of 0/1
        self.toolbox.register("attr_bool", random.randint, 0, 1)
        IND_DIMENSIONALITY = self.X_train.shape[1]
        # create template for individual
        self.toolbox.register("individual", tools.initRepeat, creator.Individual, self.toolbox.attr_bool, n=IND_DIMENSIONALITY)
        #self.toolbox.register("individual", tools.initIterate, creator.Individual, self.get_ind)
        # create template for population
        self.toolbox.register("population", tools.initRepeat, list, self.toolbox.individual)
        print('toolbox initialised')
    
    
    def initialise_fitness_function(self):
        print('building and registering the fitness function')
        
        def fitness(individual):
            
            
            accuracy,pre,fscore,recall = svm_calc_fitness(individual)
            return fscore,
            
        
        self.toolbox.register("evaluate", lambda individual: fitness(individual))
        self.toolbox.register("mate", tools.cxOnePoint)
        self.toolbox.register("mutate", tools.mutFlipBit, indpb=self.MUTPB)
        self.toolbox.register("select", tools.selTournament, tournsize=3)
        
        print('fitness function ready')
    
    def run_algorithm(self):
        
        self.initialise_toolbox()
        self.initialise_fitness_function()
        
        print('mutation probability was calculated at %f' % self.ind_mut_pb)
        print('running the algorithm')
        toolbox = self.toolbox
        # create a population of size n
        pop = toolbox.population(n=self.POPULATION_SIZE)
        
        
        for gen in range(self.NGEN):
            # Select the next generation individuals
            if gen%5 == 0:
                print(gen)
            offspring = toolbox.select(pop, len(pop))
            
            for ind in offspring:
                if ind.fitness.valid:
                    pass
            # Clone the selected individuals
            offspring = list(map(toolbox.clone, offspring))

            # Apply crossover on the offspring
            for child1, child2 in zip(offspring[::2], offspring[1::2]):
                if random.random() < self.CXPB:
                    #toolbox.mate(child1, child2)
                    tools.cxUniform(child1, child2, 0.5)
                    del child1.fitness.values
                    del child2.fitness.values

            # Apply mutation on the offspring
            for mutant in offspring:
                if random.random() < self.MUTPB:
                    # print('now doing mutation')
                    toolbox.mutate(mutant)
                    del mutant.fitness.values

            # Evaluate the individuals with an invalid fitness
            invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
            fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
            
            added = 0
            for ind, fit in zip(invalid_ind, fitnesses):
                ind.fitness.values = fit
                

            # The population is entirely replaced by the offspring
            pop[:] = offspring
            top = tools.selBest(pop, k=self.NUMBER_TO_SELECT_FROM_POP)
            
            # Dump best individual to file
            fittest = tools.selBest(pop, k=1)[0]
            count = 0
            for i in range(0, len(fittest)):
                if fittest[i] == 1:
                    count = count + 1
            with open("GA_performance.txt", "a") as text_file:
                accuracy,prec,fscore,recall = svm_calc_fitness(fittest)
                text_file.write("{} {}".format(count,fittest.fitness.values[0])+"\n")
                print(count,accuracy,prec,fscore,recall,fittest.fitness.values[0])
        
        
        
        
        max_fitness = float("-inf")
        
        for individual in tools.selBest(pop, k=1):
            if individual.fitness.valid:
                if individual.fitness.values[0] > max_fitness:
                    max_fitness = individual.fitness.values[0]
                    fittest_individual = individual
                    
        count = 0
        for i in range(0, len(fittest_individual)):
            if fittest_individual[i] == 1:
                count = count + 1        
        
        print(count, max_fitness)
        return fittest_individual

In [16]:
ga = GeneticAlgorithm(X_train, X_dev, Y_train, Y_dev)

In [17]:
fittest_individual = ga.run_algorithm()
# takes ~2s per generation (for population of size 70)

now initializing toolbox GA1
toolbox initialised
building and registering the fitness function
fitness function ready
mutation probability was calculated at 0.002940
running the algorithm
0
4193 0.8575757575757575 0.8502415458937198 0.7139959432048681 0.6153846153846154 0.7139959432048681
4193 0.8575757575757575 0.8502415458937198 0.7139959432048681 0.6153846153846154 0.7139959432048681
4252 0.8606060606060606 0.8627450980392157 0.7183673469387755 0.6153846153846154 0.7183673469387755
4209 0.8626262626262626 0.8640776699029126 0.7235772357723578 0.6223776223776224 0.7235772357723578
4204 0.8666666666666667 0.8701923076923077 0.7327935222672065 0.6328671328671329 0.7327935222672065
5
4259 0.8686868686868687 0.8861386138613861 0.7336065573770492 0.6258741258741258 0.7336065573770492
4280 0.8696969696969697 0.8651162790697674 0.7425149700598802 0.6503496503496503 0.7425149700598802
4198 0.8717171717171717 0.8994974874371859 0.7381443298969071 0.6258741258741258 0.7381443298969071
4277 0.8

4173 0.8888888888888888 0.9271844660194175 0.7764227642276423 0.6678321678321678 0.7764227642276423
4177 0.8888888888888888 0.9271844660194175 0.7764227642276423 0.6678321678321678 0.7764227642276423
4182 0.8888888888888888 0.9271844660194175 0.7764227642276423 0.6678321678321678 0.7764227642276423
4184 0.8888888888888888 0.9271844660194175 0.7764227642276423 0.6678321678321678 0.7764227642276423
85
4178 0.8888888888888888 0.9271844660194175 0.7764227642276423 0.6678321678321678 0.7764227642276423
4180 0.8888888888888888 0.9271844660194175 0.7764227642276423 0.6678321678321678 0.7764227642276423
4180 0.8888888888888888 0.9271844660194175 0.7764227642276423 0.6678321678321678 0.7764227642276423
4179 0.8888888888888888 0.9271844660194175 0.7764227642276423 0.6678321678321678 0.7764227642276423
4180 0.8888888888888888 0.9271844660194175 0.7764227642276423 0.6678321678321678 0.7764227642276423
90
4182 0.8888888888888888 0.9271844660194175 0.7764227642276423 0.6678321678321678 0.77642276422

In [18]:
fittest_individual # List[Boolean]
''.join([str(int(x)) for x in fittest_individual])

'100001000101011101010100100001011110001010110000000010011101111100101011011010000001110010110011111101000001100011011010110000000111101101101101010111100000001000001011000000001111001100100110000100111010001100100011011110111100111111000001110010101010110011111100010100110111001100010101101100010010000110001010101100000001010100100011010101010011100110111011100111011001011001100000111111100011001111011110001100100110110011111000011101100111100011000100000000100111010101101111101011001100111110010111101000101001110000011110100111011110101010000001101101011000011000010010101000100000111000011100101110010010111111110001011000100010111000101010000100001110011010001010010010101001111110001001110101011100110010110100000111000101110000111011101000001101111001100110101000110000101011101111110111010111001011001110000101110100011101000111110011000101110011010101110000100110110100100000110001011110100100011110100111000110010011111001011010111110101001011101110101010000010001001100100011001000010

In [19]:
# devset
acc, pre, fscore, recall = svm_calc_fitness(fittest_individual)
acc, pre, fscore, recall

(0.8888888888888888,
 0.9271844660194175,
 0.7764227642276423,
 0.6678321678321678)

In [20]:
# testset
acc, pre, fscore, recall = svm_calc_fitness(fittest_individual, mode='test')
acc, pre, fscore, recall

(0.8465608465608465,
 0.8548387096774194,
 0.7090301003344482,
 0.6057142857142858)

### Code Flow
- Feature extraction from the corpus (uni, bi, tri, tf-idf, lda, pos, nrc)
- GA (meta-heuristic) for feature selection
  - population of individuals, where each individual corresponds to a bitmask of features to select
  - individuals evolve over the generations by cross-over and mutation
  - fitness of individuals is evaluated using F1 score (this score is calculated on devset by fitting the model with a small amount of trainset for guiding GA)
  - after GA finishes, we end up we the fittest individual ie the optimal feature set (denoted by a bitmask) (giving a good reduction of 50%)
- Finally we train our model on the complete training set based on the selected features and evaluate it on testset